In [1]:
import pandas as pd
import numpy as np
import datetime
import gc
import math
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import re
from string import punctuation
from scipy.spatial import Voronoi


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [2]:
df_train = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv')
df_train = df_train.sort_values(by=['GameId','PlayId'])
print(df_train.shape)
display(df_train.head(200))

c:\users\0000011306852\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(509762, 49)


,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,Eric Berry,29,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,Allen Bailey,97,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,Justin Houston,50,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,Derrick Johnson,56,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,Ron Parker,38,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
5,2017090700,20170907000118,away,75.06,24.00,1.01,0.32,0.18,308.34,95.01,2543494,Dee Ford,55,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-2,252,03/19/1991,Auburn,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
6,2017090700,20170907000118,away,74.11,16.64,1.11,0.83,0.02,357.23,322.59,2543637,Terrance Mitchell,39,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,5-11,190,05/17/1992,Oregon,CB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
7,2017090700,20170907000118,away,73.37,18.73,1.24,0.74,0.13,328.52,270.04,2543851,Phillip Gaines,23,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,193,04/04/1991,Rice,CB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
8,2017090700,20170907000118,away,56.63,26.90,0.26,1.86,0.28,344.70,55.31,2550257,Daniel Sorensen,49,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-2,208,03/05/1990,Brigham Young,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
9,2017090700,20170907000118,away,73.35,38.83,4.55,0.76,0.51,75.47,190.84,2552488,Marcus Peters,22,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB

In [3]:
df_train['Position'].value_counts(dropna=False)

CB     56686
WR     53325
G      42114
T      42046
DE     35890
DT     35637
OLB    34110
TE     33237
FS     24607
C      24062
RB     23908
QB     23290
SS     20540
ILB    18541
MLB    10208
NT      8226
LB      6699
OT      6097
FB      3211
OG      3073
DB      1931
S       1641
HB       647
SAF       19
DL        17
Name: Position, dtype: int64

In [15]:
df_train.PlayId.nunique()

23171

In [14]:
a = pd.DataFrame(df_train[['PlayId','Position']].groupby('PlayId')['Position'].value_counts())
a.columns = ['count']
a = a.reset_index()
a
a[a.Position=='QB']

,PlayId,Position,count
10,20170907000118,QB,1
23,20170907000139,QB,1
36,20170907000189,QB,1
51,20170907000345,QB,1
65,20170907000395,QB,1
...,...,...,...
326539,20181230153910,QB,1
326552,20181230154035,QB,1
326566,20181230154082,QB,1
326580,20181230154135,QB,1


In [26]:
y_train = df_train[['PlayId','Yards']]
y_train = y_train.iloc[[i for i in range(y_train.shape[0]) if i%22==0]]
assert y_train.shape[0]==df_train['PlayId'].nunique()

df_train = df_train[[col for col in df_train.columns if col not in ['Yards']]]

### Some Correction
- https://www.kaggle.com/bgmello/neural-networks-feature-engineering-for-the-win

In [27]:
map_abbr = {'ARI': 'ARZ', 'BAL': 'BLT', 'CLE': 'CLV', 'HOU': 'HST'}
for abb in df_train['PossessionTeam'].unique():
    map_abbr[abb] = abb

In [28]:
def clean_WindDirection(txt):
    if pd.isna(txt):
        return np.nan
    txt = txt.lower()
    txt = ''.join([c for c in txt if c not in punctuation])
    txt = txt.replace('from', '')
    txt = txt.replace(' ', '')
    txt = txt.replace('north', 'n')
    txt = txt.replace('south', 's')
    txt = txt.replace('west', 'w')
    txt = txt.replace('east', 'e')
    return txt

def transform_WindDirection(txt):
    if pd.isna(txt):
        return np.nan
    
    if txt=='n':
        return 0
    if txt=='nne' or txt=='nen':
        return 1/8
    if txt=='ne':
        return 2/8
    if txt=='ene' or txt=='nee':
        return 3/8
    if txt=='e':
        return 4/8
    if txt=='ese' or txt=='see':
        return 5/8
    if txt=='se':
        return 6/8
    if txt=='ses' or txt=='sse':
        return 7/8
    if txt=='s':
        return 8/8
    if txt=='ssw' or txt=='sws':
        return 9/8
    if txt=='sw':
        return 10/8
    if txt=='sww' or txt=='wsw':
        return 11/8
    if txt=='w':
        return 12/8
    if txt=='wnw' or txt=='nww':
        return 13/8
    if txt=='nw':
        return 14/8
    if txt=='nwn' or txt=='nnw':
        return 15/8
    return np.nan

In [29]:
def correction_df(df):
    df['StadiumType'] = df['StadiumType'].apply(clean_StadiumType)
    df['StadiumType'] = df['StadiumType'].apply(transform_StadiumType)
    
    df['Turf'] = df['Turf'].map(Turf)
    df['Turf'] = (df['Turf'] == 'Natural') * 1
    
    df['PossessionTeam'] = df['PossessionTeam'].map(map_abbr)
    df['HomeTeamAbbr'] = df['HomeTeamAbbr'].map(map_abbr)
    df['VisitorTeamAbbr'] = df['VisitorTeamAbbr'].map(map_abbr)

    df['HomePossesion'] = (df['PossessionTeam'] == df['HomeTeamAbbr']) * 1
    df['Field_eq_Possession'] = (df['FieldPosition'] == df['PossessionTeam']) * 1
    df['HomeField'] = (df['FieldPosition'] == df['HomeTeamAbbr']) * 1
    
    t_gemeclock = pd.to_datetime(df['GameClock'])
    df['GameClock'] = t_gemeclock.dt.minute*60 + t_gemeclock.dt.second
    
    df['PlayerHeight'] = df['PlayerHeight'].map(height2cm)
    
    df['PlayerBirthDate'] = pd.to_datetime(df['PlayerBirthDate'])
    df['Age'] = 2019 - df['PlayerBirthDate'].dt.year
    
    t_handoff = pd.to_datetime(df['TimeHandoff'])
    t_handsnap = pd.to_datetime(df['TimeSnap'])
    df['TimeDelta'] = (t_handoff.dt.minute*60 + t_handoff.dt.second) - (t_handsnap.dt.minute*60 + t_handsnap.dt.second)
    
    # remove mph
    # replace the ones that has x-y by (x+y)/2
    # and also the ones with x gusts up to y
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: str(x).lower().replace('mph', '').strip() if not pd.isna(x) else x)
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
    df['WindSpeed'] = df['WindSpeed'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
    df['WindSpeed'] = df['WindSpeed'].apply(str_to_float)
    
    df['WindDirection'] = df['WindDirection'].apply(clean_WindDirection)
    df['WindDirection'] = df['WindDirection'].apply(transform_WindDirection)
    
    # binary variables to 0/1
    df['PlayDirection'] = (df['PlayDirection'].apply(lambda x: x.strip() == 'right')) * 1
    df['Team'] = (df['Team'].apply(lambda x: x.strip()=='home')) * 1
    
    df['GameWeather'] = df['GameWeather'].str.lower()
    indoor = "indoor"
    df['GameWeather'] = df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    df['GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    df['GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    df['GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    df['GameWeather'] = df['GameWeather'].apply(map_weather)
    
    df['IsRusher'] = (df['NflId'] == df['NflIdRusher']) * 1
    
    df['YardsLeft'] = df.apply(lambda row: 100-row['YardLine'] if row['HomeField'] else row['YardLine'], axis=1)
    df['YardsLeft'] = df.apply(lambda row: row['YardsLeft'] if row['PlayDirection'] else 100-row['YardsLeft'], axis=1)
    
    return df

In [30]:
def clean_StadiumType(txt):
    if pd.isna(txt):
        return np.nan
    txt = txt.lower()
    txt = ''.join([c for c in txt if c not in punctuation])
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    txt = txt.replace('outside', 'outdoor')
    txt = txt.replace('outdor', 'outdoor')
    txt = txt.replace('outddors', 'outdoor')
    txt = txt.replace('outdoors', 'outdoor')
    txt = txt.replace('oudoor', 'outdoor')
    txt = txt.replace('indoors', 'indoor')
    txt = txt.replace('ourdoor', 'outdoor')
    txt = txt.replace('retractable', 'rtr.')
    return txt

def transform_StadiumType(txt):
    if pd.isna(txt):
        return np.nan
    if 'outdoor' in txt or 'open' in txt:
        return 1
    if 'indoor' in txt or 'closed' in txt:
        return 0
    
    return np.nan

In [31]:
#from https://www.kaggle.com/c/nfl-big-data-bowl-2020/discussion/112681#latest-649087
Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 
        'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 
        'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 
        'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 
        'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'}

In [32]:
def height2cm(h):
    return int(h[0]) * 30.48 + int(h[2]) * 2.58

In [33]:
def str_to_float(txt):
    try:
        return float(txt)
    except:
        return -1

In [34]:
def map_weather(txt):
    """
    climate controlled or indoor => 3, sunny or sun => 2, clear => 1, cloudy => -1, rain => -2, snow => -3, others => 0
    partly => multiply by 0.5
    """
    ans = 1
    if pd.isna(txt):
        return 0
    if 'partly' in txt:
        ans*=0.5
    if 'climate controlled' in txt or 'indoor' in txt:
        return ans*3
    if 'sunny' in txt or 'sun' in txt:
        return ans*2
    if 'clear' in txt:
        return ans
    if 'cloudy' in txt:
        return -ans
    if 'rain' in txt or 'rainy' in txt:
        return -2*ans
    if 'snow' in txt:
        return -3*ans
    return 0

In [35]:
# プレー方向の正規化
# https://www.kaggle.com/cpmpml/initial-wrangling-voronoi-areas-in-python

def std_direction(df):

    df['ToLeft'] = df.PlayDirection == "left"
    df['IsBallCarrier'] = df.NflId == df.NflIdRusher

    df['Dir_rad'] = np.mod(90 - df.Dir, 360) * math.pi/180.0

    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"

    df['IsOnOffense'] = df.Team == df.TeamOnOffense # Is player on offense?

    df['YardLine_std'] = 100 - df.YardLine
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  'YardLine_std'] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,'YardLine']

    df['X_std'] = df.X
    df.loc[df.ToLeft, 'X_std'] = 120 - df.loc[df.ToLeft, 'X'] 
    df['Y_std'] = df.Y
    df.loc[df.ToLeft, 'Y_std'] = 160/3 - df.loc[df.ToLeft, 'Y'] 
    #train['Orientation_std'] = -90 + train.Orientation
    #train.loc[train.ToLeft, 'Orientation_std'] = np.mod(180 + train.loc[train.ToLeft, 'Orientation_std'], 360)
    df['Dir_std'] = df.Dir_rad
    df.loc[df.ToLeft, 'Dir_std'] = np.mod(np.pi + df.loc[df.ToLeft, 'Dir_rad'], 2*np.pi)
    
    
    #########################################
    # winddirection
    #########################################
    map_wind = {1/8:15/8, 2/8:14/8, 3/8:13/8, 4/8:12/8, 5/8:11/8, 6/8:10/8, 7/8:9/8}
    df['WindDirection_std'] = df.WindDirection
    df.loc[df.ToLeft, 'WindDirection_std'] =  df.loc[df.ToLeft, 'WindDirection'].map(map_wind)

    
    
    df.drop(['X','Y','Dir','YardLine','WindDirection','PlayDirection','ToLeft','IsBallCarrier','Dir_rad','TeamOnOffense','IsOnOffense'],axis=1, inplace=True)
    df = df.rename(columns={'X_std':'X','Y_std':'Y','Dir_std':'Dir','YardLine_std':'YardLine','WindDirection_std':'WindDirection'})

    return df


In [36]:
# S(Speed)のSeasonごとの差, 2017年だけ、RFIDの何かで少しおかしいらしい
# -> 2017年は2017年全体で標準正規化、2018年以降は、2018年の全体で正規化すればいい？（リークになる？）
def std_speed(df, scaler=None):
    
    if scaler is None:
        if (df.Season==2017).sum()>0:
            scaler_2017 = StandardScaler()
            scaler_2017.fit(df.loc[df.Season==2017, ['S']])
            df.loc[df.Season==2017, ['S']] = scaler_2017.transform(df.loc[df.Season==2017, ['S']].values)

        if (df.Season>2017).sum()>0:
            scaler = StandardScaler()
            scaler.fit(df.loc[df.Season==2018, ['S']])
            df.loc[df.Season>2017, ['S']] = scaler.transform(df.loc[df.Season>2017, ['S']].values)
    
    else:
        df.loc[:, ['S']] = scaler.transform(df.loc[:, ['S']].values)
    
    return df, scaler

In [37]:
def make_features(df):
    # 特徴量
    df['Dir_rad'] = np.mod(90 - df['Dir'], 360) * math.pi/180.0
    df['dx'] = df['Dis'] * np.cos(df['Dir_rad'])  # 前時刻から動いたx方向の距離
    df['dy'] = df['Dis'] * np.sin(df['Dir_rad'])  # 前時刻から動いたy方向の距離
    df['S_x'] = df['S'] * np.cos(df['Dir_rad'])  # x方向へ速度
    df['S_y'] = df['S'] * np.sin(df['Dir_rad'])  # y方向へ速度
    df['A_x'] = df['A'] * np.cos(df['Dir_rad'])  # x方向へ速度
    df['A_y'] = df['A'] * np.sin(df['Dir_rad'])  # y方向へ速度
    
    df['DefendersInTheBox_vs_Distance'] = df['DefendersInTheBox'] / df['Distance']
    
    
    # NflIdRusherを中心とした特徴量の作成
    df['IsRusher'] = (df['NflId'] == df['NflIdRusher']) * 1
    X_list = []
    for val in df[df['NflId']==df['NflIdRusher']]['X']:  # X of NflIdRusher in each PlayId
        X_temp_list = [val]*22
        X_list.extend(X_temp_list)

    Y_list = []
    for val in df[df['NflId']==df['NflIdRusher']]['Y']:  # Y of NflIdRusher in each PlayId
        Y_temp_list = [val]*22
        Y_list.extend(Y_temp_list)

    df['X_Rusher'] = X_list
    df['Y_Rusher'] = Y_list

    df['DisFromRusher'] = np.linalg.norm(df[['X_Rusher','Y_Rusher']].values-df[['X','Y']].values, axis=1)
    
    # PlayIdごとにNflIdRusherからの距離でソート
    df = df.sort_values(by=['PlayId','Team','DisFromRusher'])
    
    # Offenseかどうか
    df['IsOffense'] = 0
    df.loc[df.Team==1, 'IsOffense'] = (df['PossessionTeam']==df['HomeTeamAbbr']) *1
    df.loc[df.Team==0, 'IsOffense'] = (df['PossessionTeam']==df['VisitorTeamAbbr']) *1
    
    
    return df

In [38]:
def make_df_play_uq(df):
    # unique columns in each PlayId
    cols_play_uq = ['Season','YardLine','Quarter','GameClock','Down','Distance','HomeScoreBeforePlay','VisitorScoreBeforePlay','DefendersInTheBox','Week','StadiumType','Turf','GameWeather','Temperature','Humidity','WindSpeed','WindDirection','HomePossesion','Field_eq_Possession','HomeField','TimeDelta','YardsLeft']
    add_cols = ['DefendersInTheBox_vs_Distance']
    
    cols_play_uq.extend(add_cols) 
    cols_play_uq_cat = ['OffenseFormation','OffensePersonnel','DefensePersonnel']
    for col in cols_play_uq_cat:
        cols_play_uq.append(col)
#         for col_dum in df.columns[df.columns.str.contains('%s_'%col)]:
#             cols_play_uq.append(col_dum)

    for col in cols_play_uq:
        assert df[['PlayId',col]].groupby('PlayId')[col].nunique(dropna=False).sort_values(ascending=False).sum()==df['PlayId'].nunique()

    df_play_uq = df[['PlayId']+cols_play_uq].iloc[[i for i in range(df.shape[0]) if i%22==0]]
    assert df['PlayId'].nunique()==df_play_uq.shape[0]
    
    
    return df_play_uq, cols_play_uq_cat

In [39]:
def features_agg(df, id_col, agg_func, name):
    """
    注意として、入力のdfに対して、出力は集計されたid_colに対して集計されたdataframeを返す
    agg_func: {col名: ['mean', 'max', 'min', 'std',...], }のdict
    """
    df_agg = df.groupby(id_col).agg(agg_func)
    df_agg.columns = ['_'.join(col)+name for col in df_agg.columns.values]
    df_agg.reset_index(inplace=True)
    df_agg.fillna(0, inplace=True)

    return df_agg

In [43]:
df_train = df_train.sort_values(by=['GameId','PlayId'])    
df_train = correction_df(df_train)
df_train = std_direction(df_train)
df_train, scaler = std_speed(df_train)
df_train = make_features(df_train)
df_train

AttributeError: 'float' object has no attribute 'lower'

In [17]:
def transform_df(df, features=None, scaler=None):
    df = df.sort_values(by=['GameId','PlayId'])
    
    df = correction_df(df)
    df = std_direction(df)
    
    if scaler is None:
        df, scaler = std_speed(df)
    else:
        df, scaler = std_speed(df, scaler)

    df = make_features(df)
        
    
    ################################################
    # unique data of each play
    ################################################
    df_play_uq, cols_play_uq_cat = make_df_play_uq(df)
    
    
    ################################################
    # unique data of each player
    ################################################
    cols_player_num = ['X','Y','S','A','Dis','Orientation','Dir','PlayerHeight','PlayerWeight','Age','DisFromRusher','dx','dy','S_x','S_y','A_x','A_y']
    cols_player_cat = ['JerseyNumber','Position']
    cols_player = cols_player_num + cols_player_cat
    df_each_player = pd.DataFrame(np.array(df[cols_player]).reshape(-1,len(cols_player)*22), columns=[col+'_away_%d'%i for i in range(1,12) for col in cols_player]+[col+'_home_%d'%i for i in range(1,12) for col in cols_player])
    df_each_player['PlayId'] = df['PlayId'].unique()
    
    
    ################################################
    # Rusherに最も近いDefense, Offense
    ################################################
    df_nn_def = df[(df['IsRusher']==0) & (df['IsOffense']==0)][::11][['PlayId']+cols_player]  # Rusherに最も近いDefense
    df_nn_def.columns = [col if col=='PlayId' else col+'_NNDef' for col in df_nn_def.columns]
    df_nn_off = df[(df['IsRusher']==0) & (df['IsOffense']==1)][::10][['PlayId']+cols_player]  # Rusherに最も近いOffense
    df_nn_off.columns = [col if col=='PlayId' else col+'_NNOff' for col in df_nn_off.columns]
    
    df_nn = pd.merge(df_nn_def, df_nn_off, on='PlayId', validate='1:1')
    df_nn['DisNNDefvsNNOff'] = np.linalg.norm(df_nn[['X_NNDef','Y_NNDef']].values-df_nn[['X_NNOff','Y_NNOff']].values, axis=1)
        
    
    # 下記は時間がかかるので、要検討
    # Rusherのボロノイ領域を計算して特徴量を作成
    # まず、RusherがPlayIdの中で最も上に並ぶようにソートする
    df = df.sort_values(by=['PlayId','DisFromRusher'])
    # ソートしてから、ボロノイ領域を計算
    vor = df.groupby('PlayId')[['X','Y']].apply(Voronoi)
    
    # Rusherのボロノイ領域の面積
    def PolyArea(arr):
        x = arr[:,0]
        y = arr[:,1]
        return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))
        
    df_nn['X_maxInVoronoi'] = 0
    df_nn['X_minInVoronoi'] = 0
    df_nn['AreaInVoronoi'] = 0
    for i, playid in enumerate(list(df['PlayId'].unique())):
        df_nn.loc[df_nn['PlayId']==playid, 'X_maxInVoronoi'] = vor.iloc[i].vertices[vor.iloc[i].regions[vor.iloc[i].point_region[0]]][:,0].max()  # Rusherのボロノイ領域の最大のx座標
        df_nn.loc[df_nn['PlayId']==playid, 'X_minInVoronoi'] = vor.iloc[i].vertices[vor.iloc[i].regions[vor.iloc[i].point_region[0]]][:,0].min()  # Rusherのボロノイ領域の最小のx座標
        df_nn.loc[df_nn['PlayId']==playid, 'AreaInVoronoi'] = PolyArea(vor.iloc[i].vertices[vor.iloc[i].regions[vor.iloc[i].point_region[0]]])  # Rusherのボロノイ領域の面積
        
    
    
    ###################################################################################
    # aggreagate features per each play per offense of defense
    ###################################################################################
    # cols_playerに関するカラムの集計
    agg_player = {
        'X': ['mean','max','min','std'],
        'Y': ['mean','max','min','std'],
        'S': ['mean','max','min','std'],
        'A': ['mean','max','min','std'],
        'Dis': ['mean','max','min','std'],
        'Orientation': ['mean','max','min','std'],
        'Dir': ['mean','max','min','std'],
        'PlayerHeight': ['mean','max','min','std'],
        'PlayerWeight': ['mean','max','min','std'],
        'Age': ['mean','max','min','std'],
        'DisFromRusher': ['mean','max','min','std'],
        'dx': ['mean','max','min','std'],
        'dy': ['mean','max','min','std'],
        'S_x': ['mean','max','min','std'],
        'S_y': ['mean','max','min','std'],
        'A_x': ['mean','max','min','std'],
        'A_y': ['mean','max','min','std'],
    }
    ###################################################################################
#     # add one-hot columns
#     df = pd.get_dummies(df, columns=cols_player_cat, drop_first=True, dummy_na=True)
    
#     for col in cols_player_cat:
#         for col_dum in df.columns[df.columns.str.contains('%s_'%col)]:
#             agg_player[col_dum] = ['sum']


    df_agg_player_def = features_agg(df[df['IsOffense']==0], ['PlayId','Team'], agg_player, name='_defense').drop(['Team'], axis=1)
    df_agg_player_off = features_agg(df[df['IsOffense']==1], ['PlayId','Team'], agg_player, name='_offense').drop(['Team'], axis=1)
        
    
    # 結合
    df = pd.merge(df_play_uq, df_each_player, on='PlayId', validate='1:1')
    df = pd.merge(df, df_agg_player_def, on='PlayId', validate='1:1')
    df = pd.merge(df, df_agg_player_off, on='PlayId', validate='1:1')
    df = pd.merge(df, df_nn, on='PlayId', validate='1:1')
    
    if features is not None:
        for col in list(set(features)-set(df.columns)):
            df[col] = 0
    
    del df_play_uq, df_each_player, df_agg_player_def, df_agg_player_off, df_nn_def, df_nn_off, df_nn
    gc.collect()
    
    
    # 型変換
    df[cols_play_uq_cat] = df[cols_play_uq_cat].astype('category')
    for col in cols_player_cat:
        for col_dum in df.columns[df.columns.str.contains('%s_'%col)]:
            df[col_dum] = df[col_dum].astype('category')
    for col in cols_player_num:
        for col_dum in df.columns[df.columns.str.contains('%s_'%col)]:
            df[col_dum] = df[col_dum].astype('float')
        
    return df, scaler

In [18]:
%%time
df_train, scaler_train = transform_df(df_train)
df_train

c:\users\0000011306852\appdata\local\programs\python\python36\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Wall time: 3min 48s


,PlayId,Season,YardLine,Quarter,GameClock,Down,Distance,HomeScoreBeforePlay,VisitorScoreBeforePlay,DefendersInTheBox,Week,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection,HomePossesion,Field_eq_Possession,HomeField,TimeDelta,YardsLeft,DefendersInTheBox_vs_Distance,OffenseFormation,OffensePersonnel,DefensePersonnel,X_away_1,Y_away_1,S_away_1,A_away_1,Dis_away_1,Orientation_away_1,Dir_away_1,PlayerHeight_away_1,PlayerWeight_away_1,Age_away_1,DisFromRusher_away_1,dx_away_1,dy_away_1,S_x_away_1,S_y_away_1,A_x_away_1,A_y_away_1,JerseyNumber_away_1,Position_away_1,X_away_2,Y_away_2,S_away_2,A_away_2,Dis_away_2,Orientation_away_2,Dir_away_2,PlayerHeight_away_2,PlayerWeight_away_2,Age_away_2,DisFromRusher_away_2,dx_away_2,dy_away_2,S_x_away_2,S_y_away_2,A_x_away_2,A_y_away_2,JerseyNumber_away_2,Position_away_2,X_away_3,Y_away_3,S_away_3,A_away_3,Dis_away_3,Orientation_away_3,Dir_away_3,PlayerHeight_away_3,PlayerWeight_away_3,Age_away_3,DisFromRusher_away_3,dx_away_3,dy_away_3,S_x_away_3,S_y_away_3,A_x_away_3,A_y_away_3,JerseyNumber_away_3,Position_away_3,X_away_4,Y_away_4,S_away_4,A_away_4,Dis_away_4,Orientation_away_4,Dir_away_4,PlayerHeight_away_4,PlayerWeight_away_4,Age_away_4,DisFromRusher_away_4,dx_away_4,dy_away_4,S_x_away_4,S_y_away_4,A_x_away_4,A_y_away_4,JerseyNumber_away_4,Position_away_4,X_away_5,Y_away_5,S_away_5,A_away_5,Dis_away_5,Orientation_away_5,Dir_away_5,PlayerHeight_away_5,PlayerWeight_away_5,Age_away_5,DisFromRusher_away_5,dx_away_5,dy_away_5,S_x_away_5,S_y_away_5,A_x_away_5,A_y_away_5,JerseyNumber_away_5,Position_away_5,X_away_6,Y_away_6,S_away_6,A_away_6,Dis_away_6,Orientation_away_6,Dir_away_6,PlayerHeight_away_6,PlayerWeight_away_6,Age_away_6,DisFromRusher_away_6,dx_away_6,dy_away_6,S_x_away_6,S_y_away_6,A_x_away_6,A_y_away_6,JerseyNumber_away_6,Position_away_6,X_away_7,Y_away_7,S_away_7,A_away_7,Dis_away_7,Orientation_away_7,Dir_away_7,PlayerHeight_away_7,PlayerWeight_away_7,Age_away_7,DisFromRusher_away_7,dx_away_7,dy_away_7,S_x_away_7,S_y_away_7,A_x_away_7,A_y_away_7,JerseyNumber_away_7,Position_away_7,X_away_8,Y_away_8,S_away_8,A_away_8,Dis_away_8,Orientation_away_8,Dir_away_8,PlayerHeight_away_8,PlayerWeight_away_8,Age_away_8,DisFromRusher_away_8,dx_away_8,dy_away_8,S_x_away_8,S_y_away_8,A_x_away_8,A_y_away_8,JerseyNumber_away_8,Position_away_8,X_away_9,Y_away_9,S_away_9,A_away_9,Dis_away_9,Orientation_away_9,Dir_away_9,PlayerHeight_away_9,PlayerWeight_away_9,Age_away_9,DisFromRusher_away_9,dx_away_9,dy_away_9,S_x_away_9,S_y_away_9,A_x_away_9,A_y_away_9,JerseyNumber_away_9,Position_away_9,X_away_10,Y_away_10,S_away_10,A_away_10,Dis_away_10,Orientation_away_10,Dir_away_10,PlayerHeight_away_10,PlayerWeight_away_10,Age_away_10,DisFromRusher_away_10,dx_away_10,dy_away_10,S_x_away_10,S_y_away_10,A_x_away_10,A_y_away_10,JerseyNumber_away_10,Position_away_10,X_away_11,Y_away_11,S_away_11,A_away_11,Dis_away_11,Orientation_away_11,Dir_away_11,PlayerHeight_away_11,PlayerWeight_away_11,Age_away_11,DisFromRusher_away_11,dx_away_11,dy_away_11,S_x_away_11,S_y_away_11,A_x_away_11,A_y_away_11,JerseyNumber_away_11,Position_away_11,X_home_1,Y_home_1,S_home_1,A_home_1,Dis_home_1,Orientation_home_1,Dir_home_1,PlayerHeight_home_1,PlayerWeight_home_1,Age_home_1,DisFromRusher_home_1,dx_home_1,dy_home_1,S_x_home_1,S_y_home_1,A_x_home_1,A_y_home_1,JerseyNumber_home_1,Position_home_1,X_home_2,Y_home_2,S_home_2,A_home_2,Dis_home_2,Orientation_home_2,Dir_home_2,PlayerHeight_home_2,PlayerWeight_home_2,Age_home_2,DisFromRusher_home_2,dx_home_2,dy_home_2,S_x_home_2,S_y_home_2,A_x_home_2,A_y_home_2,JerseyNumber_home_2,Position_home_2,X_home_3,Y_home_3,S_home_3,A_home_3,Dis_home_3,Orientation_home_3,Dir_home_3,PlayerHeight_home_3,PlayerWeight_home_3,Age_home_3,DisFromRusher_home_3,dx_home_3,dy_home_3,S_x_home_3,S_y_home_3,A_x_home_3,A_y_home_3,JerseyNumber_home_3,Position_home_3,X_home_4,Y_home_4,S_home_4,A_home_4,Dis_home_4,Orientation_home_4,Dir_home_4,PlayerHeight_home_4,PlayerWeight_home_4,Age_home_4,DisFrom

In [19]:
cols_rm = ['PlayId','Yards','Season']
features = [col for col in df_train.columns if col not in cols_rm]


df_train = pd.merge(df_train, y_train, on='PlayId', validate='1:1')
X_train = df_train[features]
y_train = df_train['Yards'] + 99

In [22]:
params = {
    'objective':'multiclass',
    'num_class': 199,
    "boosting": "gbdt",
    #"metric": 'mae',
    'learning_rate': 0.01,
    'num_leaves': 63,
    'max_depth': -1,
    # 'min_data_in_leaf': 30, 
    # "min_child_samples": 20,
    # "feature_fraction": 0.9,
    # "bagging_freq": 1,
    # "bagging_fraction": 0.9 ,
    # "bagging_seed": 11,
    # "lambda_l1": 0.1,
    "verbosity": -1,
    "nthread": -1,
    "random_state": 16
}

In [23]:
from sklearn.model_selection import RepeatedKFold, KFold, StratifiedKFold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=16)

models = []
oof = np.zeros((509762//22, 199))
feature_importance_df = pd.DataFrame()
for fold_, (tr_idx, val_idx) in enumerate(folds.split(X_train.values, y_train.values)):
    print('-')
    print("Fold {}".format(fold_ + 1))
    
    tr_data = lgb.Dataset(X_train.iloc[tr_idx], label=y_train[tr_idx])
    val_data = lgb.Dataset(X_train.iloc[val_idx], label=y_train[val_idx])

    num_round = 10000
    model = lgb.train(params, tr_data, num_round, valid_sets = [tr_data, val_data], verbose_eval=100, early_stopping_rounds=100)
    oof[val_idx, :] = model.predict(X_train.iloc[val_idx], num_iteration=model.best_iteration)
    
    models.append(model)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = model.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

c:\users\0000011306852\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


-
Fold 1
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 1.66414	valid_1's multi_logloss: 2.89626


KeyboardInterrupt: 

In [ ]:
oof

In [ ]:
y_ans = np.zeros((509762//22, 199))
for i,p in enumerate(y_train-99):
    p+=99
    for j in range(199):
        if j>=p:
            y_ans[i][j]=1.0

print("validation score:",np.sum(np.power(np.cumsum(oof, axis=1)-y_ans, 2))/(199*(509762//22)))

In [ ]:
# output importance
feature_importance_mean = feature_importance_df[['feature','importance']].groupby('feature').mean().sort_values(by="importance", ascending=False)
print(feature_importance_mean.head(20))
feature_importance_mean.to_csv("../importance/lgbm_2.csv", index=True)

In [ ]:
feature_importance_mean[feature_importance_mean['importance']==0].index

In [ ]:
with open("../importance/null_features_2.txt", 'wt') as f:
    for ele in list(feature_importance_mean[feature_importance_mean['importance']==0].index):
        f.write('"'+ele+'",'+'\n')

In [ ]:
y=np.array([[0.5,0.6,0.7,0.55,0.45,0.77],[0.5,0.2,0.3,0.55,0.45,0.77],[0.5,0.6,0.7,0.45,0.35,0.77]])
y = np.mean(y, axis=0)
print(y)
y = np.cumsum(y, axis=0).clip(0,1)
y

In [ ]:
ab = np.cumsum(0.4, axis=0)#.clip(0, 1)
aa+=ab
aa

In [ ]:
def make_pred(df, sample, env, models):
    df, scaler_train = transform_df(df, scaler_train)
    
    for col in list(set(df_train) - set(df)):
        df[col] = 0
    df = df[features]
    
    y_pred = np.mean([model.predict(df) for model in models], axis=0)
    y_pred = np.cumsum(y_pred, axis=0).clip(0, 1)
    for pred in y_pred:
        prev=0
        for i in range(len(pred)):
            if prev>pred[i]:
                pred[i]=prev
            prev=pred[i]
    yardsleft = np.array(df['YardsLeft'][::22])
    
    for i in range(len(yardsleft)):
        y_pred[i, :yardsleft[i]-1] = np.zeros(shape=y_pred[i, :yardsleft[i]-1].shape)
        y_pred[i, yardsleft[i]+100:] = np.ones(shape=y_pred[i, yardsleft[i]+100:].shape)
    env.predict(pd.DataFrame(data=y_pred, columns=sample.columns))
    return y_pred

In [ ]:
for (test, sample) in env.iter_test():
    make_pred(test, sample, env, models)
env.write_submission_file()